# 컬렉션 만들기

- 컬렉션은 고정 열과 변형 행이 있는 2차원 테이블
- 각 열은 필드를 나타내고 각 행은 엔티티를 나타냄
- 컬렉션을 만들려면 다음과 같은 사항을 수행해야함
    - 스키마 만들기
    - 인덱스 매개변수 설정 (선택 사항)
    - 컬렉션 만들기

참고자료: https://milvus.io/docs/ko/create-collection.md

## 스키마 만들기

- 스키마는 컬렉션의 데이터 구조를 정의하며, 컬렉션을 만들 때는 요구 사항에 따라 스키마를 설계해야함

In [21]:
from pymilvus import MilvusClient, DataType

client = MilvusClient(
    uri="http://localhost:19530",
    token="root:Milvus"
)

In [22]:
# 3.1. Create schema
schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)

In [23]:
# 3.2. Add fields to schema
schema.add_field(field_name="my_id", datatype=DataType.INT64, is_primary=True) # primary 키로 동작
schema.add_field(field_name="my_vector", datatype=DataType.FLOAT_VECTOR, dim=5) # 벡터 필드 # 임베딩의 dim으로 표시
schema.add_field(field_name="my_varchar", datatype=DataType.VARCHAR, max_length=512) # 문자열 필드

{'auto_id': False, 'description': '', 'fields': [{'name': 'my_id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'my_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 5}}, {'name': 'my_varchar', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}], 'enable_dynamic_field': True}

## (선택 사항) 인덱스 매개변수 설정

- 특정 필드에 인덱스를 생성하면 이 필드에 대한 검색 속도가 빨라짐 -> 인덱스는 컬렉션 내 엔티티의 순서를 기록
- metric_type 및 index_type을 사용하여 Milvus가 필드를 색인하고 벡터 임베딩 간의 유사성을 측정하는 적절한 방법을 선택할 수 있음
- Milvus에서는 모든 벡터 필드에 대한 인덱스 유형으로 AUTOINDEX을 사용하고 필요에 따라 COSINE, L2, IP 중 하나를 메트릭 유형으로 사용 가능
- Vector 필드에서 Index는 필수적이며, 필터링 조건에서 자주 사용되는 스칼라 필드에 Index를 생성하는 것이 좋음

### 참고자료
- 벡터 필드 색인 생성: https://milvus.io/docs/ko/index-vector-fields.md?tab=floating
- 스칼라 필드 색인 생성: https://milvus.io/docs/ko/index-scalar-fields.md

In [24]:
# 3.3. Prepare index parameters
index_params = client.prepare_index_params()

# 3.4. Add indexes
index_params.add_index(
    field_name="my_id",
    index_type="AUTOINDEX"
)

index_params.add_index(
    field_name="my_vector",
    index_type="AUTOINDEX",
    metric_type="COSINE"
)

In [25]:
index_params

[{'field_name': 'my_id', 'index_type': 'AUTOINDEX', 'index_name': ''},
 {'field_name': 'my_vector', 'index_type': 'AUTOINDEX', 'index_name': '', 'metric_type': 'COSINE'}]

## 컬렉션 만들기
- 인덱스 파라미터가 있는 컬렉션을 생성한 경우, Milvus는 생성 시 자동으로 컬렉션을 로드
- 이 경우 인덱스 매개변수에 언급된 모든 필드가 인덱싱 됨

In [27]:
# 3.5. Create a collection with the index loaded simultaneously
client.create_collection(
    collection_name="customized_setup__1",
    schema=schema,
    index_params=index_params
)

# client.load_collection("customized_setup__1")

res = client.get_load_state(
    collection_name="customized_setup__1"
)

print(res)

{'state': <LoadState: Loaded>}


In [28]:
# 3.6. Create a collection and index it separately

client.create_collection(
    collection_name="customized_setup_2",
    schema=schema,
)

# client.load_collection("customized_setup__2")

res = client.get_load_state(
    collection_name="customized_setup_2"
)

print(res)

# Output
#
# {
#     "state": "<LoadState: NotLoad>"
# }


{'state': <LoadState: NotLoad>}


# 컬렉션 속성 설정

- 생성할 컬렉션의 속성을 설정하여 서비스에 적합하게 만듦

## 샤드 번호 설정
- 샤드(Shard)란?
    - Milvus에서 데이터를 분산 저장하는 논리적 단위
    - 하나의 컬렉션에서 여러 샤드로 나뉘어 저장됨
    - 각 샤드는 독립적으로 데이터를 처리하고 검색을 수행
- 일반적인 경우, 예상 처리량이 500MB/s 증가하거나 삽입할 데이터의 양이 100GB 증가할 때마다 샤드 수를 하나씩 늘리는 것을 고려

In [29]:
# With shard Number
client.create_collection(
    collection_name="customized_setup_3",
    schema=schema,
    num_shards=1
)

## mmap 활성화
- Milvus는 기본적으로 모든 컬렉션에서 mmap을 활성화하여 원시 필드 데이터를 완전히 로드하는 대신 메모리에 매핑할 수 있도록 함
- 이렇게 하면 메모리 사용량이 줄어들고 수집용량이 증가

In [31]:
# With mmap
client.create_collection(
    collection_name="customized_setup_4",
    schema=schema,
    enable_map=False
)

## 컬랙션 TTL 설정

In [32]:
# With TTL
client.create_collection(
    collection_name="customized_setup_5",
    schema=schema,
    properties={
        "collection.ttl.seconds": 86400
    }
)

## 일관성 수준 설정

In [33]:
# With consistency level
client.create_collection(
    collection_name="customized_setup_6",
    schema=schema,
    consistency_level="Bounded",
)

# 컬렉션 보기

In [34]:
from pymilvus import MilvusClient

client = MilvusClient(
    uri="http://localhost:19530",
    token="root:Milvus"
)

res = client.list_collections()